In [36]:
from pathlib import Path
import os
import numpy as np
import pandas as pd

folder = Path(os.environ["TMPDIR"]) / "Detect dataset"
folder

PosixPath('/run/nvme/job_18489501/tmp/Detect dataset')

In [111]:
img_folder_og = folder / "Original images"

cols = [0,1,2,3,4,5,6,7,8,9, "data_partitions", "im_num", "im_path", "cls_name", "cls_id"]
df = {}
df[1] = pd.read_csv(folder / "dataset1_partitions.txt", skiprows=1, sep=" ", names=cols)
df[2] = pd.read_csv(folder / "dataset2_partitions.txt", skiprows=1, sep=" ", names=cols)
df[3] = pd.read_csv(folder / "dataset3_partitions.txt", skiprows=1, sep=" ", names=cols)
df[2]

,0,1,2,3,4,5,6,7,8,9,data_partitions,im_num,im_path,cls_name,cls_id
0,val,train,train,test,test,train,train,train,val,train,2,Ephemerella_aroni_aurivillii\\100_%s.png,Ephemerella_aroni_aurivillii,1,NaN
1,train,train,train,train,val,train,train,train,train,val,2,Ephemerella_aroni_aurivillii\\102_%s.png,Ephemerella_aroni_aurivillii,1,NaN
2,train,train,train,test,train,test,test,train,train,test,2,Ephemerella_aroni_aurivillii\\103_%s.png,Ephemerella_aroni_aurivillii,1,NaN
3,val,train,test,train,test,train,test,test,train,val,2,Ephemerella_aroni_aurivillii\\104_%s.png,Ephemerella_aroni_aurivillii,1,NaN
4,test,train,train,train,test,val,train,test,train,val,2,Ephemerella_aroni_aurivillii\\105_%s.png,Ephemerella_aroni_aurivillii,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6033,test,test,train,train,test,train,train,train,val,train,2,Atherix_ibis\\94_%s.png,Atherix_ibis,29,NaN
6034,test,test,val,test,val,train,test,train,test,test,2,Atherix_ibis\\95_%s.png,Atherix_ibis,29,NaN
6035,train,val,test,train,train,train,train,train,test,val,2,Atherix_ibis\\96_%s.png,Atherix_ibis,29,NaN
6036,val,train,train,test,test,test,train,train,test,train,2,Atherix_ibis\\97_%s.png,Atherix_ibis,29,NaN


In [112]:
all_taxons_og = [f.name for f in img_folder_og.glob("*") if f.is_dir()]
all_taxons_og

['Agrypnia_sp',
 'Ameletus_inopinatus',
 'Asellus_aquaticus',
 'Baetis_digitatus',
 'Baetis_niger',
 'Baetis_rhodani',
 'Brachyptera_risi',
 'Callicorixa_wollastoni',
 'Centroptilum_luteolum',
 'Ceraclea_excisa',
 'Ceratopogonidae',
 'Chelifera',
 'Chimarra_marginata',
 'Chironomidae',
 'Cyrnus_flavidus',
 'Dicranota',
 'Diura_nanseni',
 'Elmis_aenea_adult',
 'Elmis_aenea_larva',
 'Elodes',
 'Eloeophila_sp',
 'Ephemerella_aroni_aurivillii',
 'Gammarus_lacustris',
 'Gyraulus_sp',
 'Habrophlebia_sp',
 'Heptagenia_dalecarlica',
 'Hydracarina',
 'Hydraena_adult',
 'Hydropsyche_pellucidula',
 'Hydropsyche_saxonica',
 'Hydropsyche_siltalai',
 'Isoperla_sp',
 'Ithytrichia_lamellaris',
 'Lepidostoma_hirtum',
 'Leptophlebia_sp',
 'Leuctra_nigra',
 'Leuctra_sp',
 'Limnephilidae',
 'Limnius_volckmari_adult',
 'Micrasema_gelidum',
 'Micrasema_setiferum',
 'Nemoura_sp',
 'Neureclipsis_bimaculata',
 'Oulimnius_tuberculatus_adult',
 'Oulimnius_tuberculatus_larva',
 'Oxyethira_sp',
 'Paraleptophlebia_

In [113]:
# the dataframe has some taxons that are not present in the folder

taxons = set()
for key, dfp in df.items():
    taxons = taxons | set(dfp.im_path.unique())
print(taxons - set(all_taxons_og))
print(set(all_taxons) - taxons)

{'Atherix_ibis', 'Hemerodromia'}
set()


In [114]:
# These samples are in the first two datasets

for key, dfp in df.items():
    r = dfp[dfp.im_path.isin(taxons - set(all_taxons_og))]
    print(key, len(r), r.im_path.unique())

1 263 ['Hemerodromia' 'Atherix_ibis']
2 263 ['Hemerodromia' 'Atherix_ibis']
3 0 []


In [116]:
# This problem does not exist with the 'Cropped images' Folder
img_folder = folder / "Cropped images"

all_taxons = [f.name for f in img_folder.glob("*") if f.is_dir()]

taxons = set()
for key, dfp in df.items():
    taxons = taxons | set(dfp.im_path.unique())
print(taxons - set(all_taxons))
print(set(all_taxons) - taxons)

# So we will use the cropped images

set()
set()


In [268]:
filelist = []
for taxon in all_taxons:
    taxon_folder = img_folder / taxon
    files = list(taxon_folder.glob("*.png"))
    fdf = pd.DataFrame({"fname": files,
                        "im_path": taxon})

    fdf = fdf.assign(im_num_true = fdf["fname"].apply(lambda x: x.stem.split("_")[0]),
                     fname = fdf["fname"].apply(lambda x: x.name))
    filelist.append(fdf)
file_df = pd.concat(filelist).set_index(["im_path", "im_num_true"]).sort_index()
file_df

fname
im_path               im_num_true          
Agrypnia_sp           12           12_l.png
                      12           12_r.png
                      14           14_l.png
                      14           14_r.png
                      19           19_l.png
...                                     ...
Taeniopteryx_nebulosa 93           93_r.png
                      96           96_l.png
                      96           96_r.png
                      99           99_l.png
                      99           99_r.png

[15074 rows x 1 columns]

In [278]:
def process_dataset_partition(dfp):
    dfp = dfp.assign(fname=dfp["im_num"].apply(lambda x: x.split("\\")[2]))
    dfp = dfp.assign(im_num_true=dfp.fname.apply(lambda x: x.split("_")[0]))
    dfp = dfp.set_index(["im_path", "im_num_true"]).sort_index()
    
    filtered_df = file_df.loc[dfp.index].reset_index()
    filtered_df = filtered_df.rename(columns={"im_path": "taxon",
                                              "im_num_true": "individual",
                                              "fname": "img"})
    return filtered_df

process_dataset_partition(df[2])

,taxon,individual,img
0,Ameletus_inopinatus,1,1_l.png
1,Ameletus_inopinatus,1,1_r.png
2,Ameletus_inopinatus,10,10_l.png
3,Ameletus_inopinatus,10,10_r.png
4,Ameletus_inopinatus,100,100_l.png
...,...,...,...
11827,Taeniopteryx_nebulosa,93,93_r.png
11828,Taeniopteryx_nebulosa,96,96_l.png
11829,Taeniopteryx_nebulosa,96,96_r.png
11830,Taeniopteryx_nebulosa,99,99_l.png
